# Análisis exploratorio: Dataset "550K Spotify Songs: Audio, Lyrics & Genres"
Exploraré sobre los generos, seguidores y popularidad de artistas que suelo escuchar y revisar también el comportamiento de escucha por generos como "corridos", "regional", etc. 

## Rutas e importaciones

In [49]:
import pandas as pd
import plotly.express as px

In [50]:
raw_df_spoty = pd.read_csv('../data/raw/artists.csv')
raw_df_spoty.to_parquet('../data/processed/raw_data_spoty.parquet')
raw_df_spoty.head()

,id,name,followers,popularity,genres,main_genre
0,6YROFUbu5zRCHi2xkir5pk,Brian Hyland,67223,47,[],Pop
1,5tFRohaO5yEsuJxmMnlCO9,Barns Courtney,602647,62,[],Electronic
2,3w1Q754jb31h5CXQCcnLNL,Capcom Sound Team,210392,58,"['japanese vgm', 'soundtrack']",Electronic
3,3oDbviiivRWhXwIE8hxkVV,The Beach Boys,5139194,76,['baroque pop'],Classical
4,60zvRmhQHRxokEB1taAVpN,Beth Malone,1569,29,['musicals'],Classical


## Inspección general

In [51]:
raw_df_spoty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71440 entries, 0 to 71439
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          71440 non-null  object
 1   name        71438 non-null  object
 2   followers   71440 non-null  int64 
 3   popularity  71440 non-null  int64 
 4   genres      71440 non-null  object
 5   main_genre  71440 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.3+ MB


**Estandarizacón de strings**

In [52]:
raw_df_spoty.columns = raw_df_spoty.columns.str.lower().str.strip().str.replace(' ', '_')
raw_df_spoty.columns

Index(['id', 'name', 'followers', 'popularity', 'genres', 'main_genre'], dtype='object')

In [53]:
# df limpio a parquet
spotify_data = raw_df_spoty.copy()
spotify_data.to_parquet('../data/processed/spotify_data_ok.parquet')

## Agrupaciones

**Edición de los beatles porque están sobre valorados**

In [54]:
spotify_data.loc[
    spotify_data['name'] == 'The Beatles', 
    ['followers', 'popularity']
    ] = [0, 0]
spotify_data.loc[spotify_data['name'] == 'The Beatles']

,id,name,followers,popularity,genres,main_genre
6029,3WrFJ7ztbogyGnTHbHJFl2,The Beatles,0,0,"['classic rock', 'psychedelic rock']",Rock


In [55]:
genero_princ = spotify_data['main_genre'].str.contains('corridos')
spotify_data[genero_princ].head()

,id,name,followers,popularity,genres,main_genre


In [61]:
data_por_gen = (
    spotify_data
    .groupby('main_genre')
    .agg(
        seguidores_por_gen = ('followers', 'sum'),
        popularidad_por_gen = ('popularity', 'mean')
        )
    .reset_index()
    )
met_por_genero = px.bar(
    data_por_gen, 
    x='main_genre', 
    y='seguidores_por_gen',
    title='Total de seguidores por género',
    text='seguidores_por_gen'
    )
met_por_genero.show()